In [20]:
#install.packages("singleCellHaystack")

In [21]:
getwd()
setwd('/data01/ch6845/MarcoPolo/')
getwd()

[1] "/data01/ch6845/MarcoPolo"

[1] "/data01/ch6845/MarcoPolo"

In [10]:
library(singleCellHaystack)

In [11]:
set.seed(1234)

In [12]:
library(Matrix)
library(Seurat)
library(mclust)
#library(clustree)
#citation("mclust")

In [13]:
read_in_seurat <-function(save_name,method){
    exp_data=readMM(sprintf("datasets/extract/%s.data.counts.mm",save_name))
 
    exp_data_row=read.table(sprintf('datasets/extract/%s.data.row',save_name))$V1
    exp_data_col=read.table(sprintf('datasets/extract/%s.data.col',save_name))$V1    
    rownames(exp_data)=exp_data_row
    colnames(exp_data)=exp_data_col    

    metadatarow=read.table(sprintf('datasets/extract/%s.metadatarow.tsv',save_name),sep='\t')
    metadatacol=read.table(sprintf('datasets/extract/%s.metadatacol.tsv',save_name),sep='\t')  
    
    metadatacol['size_factor']=read.table(sprintf('datasets/extract/%s.size_factor.tsv',save_name),sep='\t')$V1

    seurat <- CreateSeuratObject(counts = exp_data, project = "scRNAseq", assay = "RNA",
                                         min.cells = 0, min.features = 0,
                                         meta.data = as.data.frame(metadatacol))    
    
    for(col in colnames(metadatarow)){
        #print(col)
        #print(head(metadatarow[rownames(seurat[['RNA']]@meta.features),][col]))
        seurat[['RNA']]@meta.features[col]=metadatarow[rownames(seurat[['RNA']]@meta.features),][col]
    }
    seurat
}

In [14]:
dataset_name_all=list(
'Kohinbulk_filtered',
'HumanLiver_filtered',
'Zhengmix8eq_filtered')

length(dataset_name_all)

[1] 3

In [15]:
dataset_name_all=list()
for (ncells_total in c('1000','2000','5000','10000')){
    for (prop in c('1e-2','5e-3','1e-3','5e-4')){
        for(i in c(1:10)){
            dataset_name<-sprintf('Simul_%s_%s_%s_filtered',ncells_total,prop,i)
            dataset_name_all<-append(dataset_name_all,dataset_name)
        }

    }
}

In [16]:
tabula_name_all=list("TabulaAorta_filtered",
"TabulaBladder_filtered",
"TabulaBrainMyeloid_filtered",
"TabulaBrainNonMyeloid_filtered",
"TabulaDiaphragm_filtered",
"TabulaFat_filtered",
"TabulaHeart_filtered",
"TabulaKidney_filtered",
"TabulaLargeIntestine_filtered",
"TabulaLimbMuscle_filtered",
"TabulaLiver_filtered",
"TabulaLung_filtered",
"TabulaMammaryGland_filtered",
"TabulaMarrow_filtered",
"TabulaPancreas_filtered",
"TabulaSkin_filtered",
"TabulaSpleen_filtered",
"TabulaThymus_filtered",
"TabulaTongue_filtered",
"TabulaTrachea_filtered")

In [ ]:
#taskset -c 12,13,14,15 Rscript 4_exp1_step2_clustering_1-Copy2.r
#taskset -c 16,17,18,19 Rscript 4_exp1_step2_clustering_1-Copy3.r

In [17]:
head(dataset_name_all)

[[1]]
[1] "Simul_1000_1e-2_1_filtered"

[[2]]
[1] "Simul_1000_1e-2_2_filtered"

[[3]]
[1] "Simul_1000_1e-2_3_filtered"

[[4]]
[1] "Simul_1000_1e-2_4_filtered"

[[5]]
[1] "Simul_1000_1e-2_5_filtered"

[[6]]
[1] "Simul_1000_1e-2_6_filtered"

In [18]:
dataset_name_all

[[1]]
[1] "Simul_1000_1e-2_1_filtered"

[[2]]
[1] "Simul_1000_1e-2_2_filtered"

[[3]]
[1] "Simul_1000_1e-2_3_filtered"

[[4]]
[1] "Simul_1000_1e-2_4_filtered"

[[5]]
[1] "Simul_1000_1e-2_5_filtered"

[[6]]
[1] "Simul_1000_1e-2_6_filtered"

[[7]]
[1] "Simul_1000_1e-2_7_filtered"

[[8]]
[1] "Simul_1000_1e-2_8_filtered"

[[9]]
[1] "Simul_1000_1e-2_9_filtered"

[[10]]
[1] "Simul_1000_1e-2_10_filtered"

[[11]]
[1] "Simul_1000_5e-3_1_filtered"

[[12]]
[1] "Simul_1000_5e-3_2_filtered"

[[13]]
[1] "Simul_1000_5e-3_3_filtered"

[[14]]
[1] "Simul_1000_5e-3_4_filtered"

[[15]]
[1] "Simul_1000_5e-3_5_filtered"

[[16]]
[1] "Simul_1000_5e-3_6_filtered"

[[17]]
[1] "Simul_1000_5e-3_7_filtered"

[[18]]
[1] "Simul_1000_5e-3_8_filtered"

[[19]]
[1] "Simul_1000_5e-3_9_filtered"

[[20]]
[1] "Simul_1000_5e-3_10_filtered"

[[21]]
[1] "Simul_1000_1e-3_1_filtered"

[[22]]
[1] "Simul_1000_1e-3_2_filtered"

[[23]]
[1] "Simul_1000_1e-3_3_filtered"

[[24]]
[1] "Simul_1000_1e-3_4_filtered"

[[25]]
[1] "Simul_1000_1e-3_5_filtered"

[[26]]
[1] "Simul_1000_1e-3_6_filtered"

[[27]]
[1] "Simul_1000_1e-3_7_filtered"

[[28]]
[1] "Simul_1000_1e-3_8_filtered"

[[29]]
[1] "Simul_1000_1e-3_9_filtered"

[[30]]
[1] "Simul_1000_1e-3_10_filtered"

[[31]]
[1] "Simul_1000_5e-4_1_filtered"

[[32]]
[1] "Simul_1000_5e-4_2_filtered"

[[33]]
[1] "Simul_1000_5e-4_3_filtered"

[[34]]
[1] "Simul_1000_5e-4_4_filtered"

[[35]]
[1] "Simul_1000_5e-4_5_filtered"

[[36]]
[1] "Simul_1000_5e-4_6_filtered"

[[37]]
[1] "Simul_1000_5e-4_7_filtered"

[[38]]
[1] "Simul_1000_5e-4_8_filtered"

[[39]]
[1] "Simul_1000_5e-4_9_filtered"

[[40]]
[1] "Simul_1000_5e-4_10_filtered"

[[41]]
[1] "Simul_2000_1e-2_1_filtered"

[[42]]
[1] "Simul_2000_1e-2_2_filtered"

[[43]]
[1] "Simul_2000_1e-2_3_filtered"

[[44]]
[1] "Simul_2000_1e-2_4_filtered"

[[45]]
[1] "Simul_2000_1e-2_5_filtered"

[[46]]
[1] "Simul_2000_1e-2_6_filtered"

[[47]]
[1] "Simul_2000_1e-2_7_filtered"

[[48]]
[1] "Simul_2000_1e-2_8_filtered"

[[49]]
[1] "Simul_2000_1e-2_9_filtered"

[[50]]
[1] "Simul_2000_1e-2_10_filtered"

[[51]]
[1] "Simul_2000_5e-3_1_filtered"

[[52]]
[1] "Simul_2000_5e-3_2_filtered"

[[53]]
[1] "Simul_2000_5e-3_3_filtered"

[[54]]
[1] "Simul_2000_5e-3_4_filtered"

[[55]]
[1] "Simul_2000_5e-3_5_filtered"

[[56]]
[1] "Simul_2000_5e-3_6_filtered"

[[57]]
[1] "Simul_2000_5e-3_7_filtered"

[[58]]
[1] "Simul_2000_5e-3_8_filtered"

[[59]]
[1] "Simul_2000_5e-3_9_filtered"

[[60]]
[1] "Simul_2000_5e-3_10_filtered"

[[61]]
[1] "Simul_2000_1e-3_1_filtered"

[[62]]
[1] "Simul_2000_1e-3_2_filtered"

[[63]]
[1] "Simul_2000_1e-3_3_filtered"

[[64]]
[1] "Simul_2000_1e-3_4_filtered"

[[65]]
[1] "Simul_2000_1e-3_5_filtered"

[[66]]
[1] "Simul_2000_1e-3_6_filtered"

[[67]]
[1] "Simul_2000_1e-3_7_filtered"

[[68]]
[1] "Simul_2000_1e-3_8_filtered"

[[69]]
[1] "Simul_2000_1e-3_9_filtered"

[[70]]
[1] "Simul_2000_1e-3_10_filtered"

[[71]]
[1] "Simul_2000_5e-4_1_filtered"

[[72]]
[1] "Simul_2000_5e-4_2_filtered"

[[73]]
[1] "Simul_2000_5e-4_3_filtered"

[[74]]
[1] "Simul_2000_5e-4_4_filtered"

[[75]]
[1] "Simul_2000_5e-4_5_filtered"

[[76]]
[1] "Simul_2000_5e-4_6_filtered"

[[77]]
[1] "Simul_2000_5e-4_7_filtered"

[[78]]
[1] "Simul_2000_5e-4_8_filtered"

[[79]]
[1] "Simul_2000_5e-4_9_filtered"

[[80]]
[1] "Simul_2000_5e-4_10_filtered"

[[81]]
[1] "Simul_5000_1e-2_1_filtered"

[[82]]
[1] "Simul_5000_1e-2_2_filtered"

[[83]]
[1] "Simul_5000_1e-2_3_filtered"

[[84]]
[1] "Simul_5000_1e-2_4_filtered"

[[85]]
[1] "Simul_5000_1e-2_5_filtered"

[[86]]
[1] "Simul_5000_1e-2_6_filtered"

[[87]]
[1] "Simul_5000_1e-2_7_filtered"

[[88]]
[1] "Simul_5000_1e-2_8_filtered"

[[89]]
[1] "Simul_5000_1e-2_9_filtered"

[[90]]
[1] "Simul_5000_1e-2_10_filtered"

[[91]]
[1] "Simul_5000_5e-3_1_filtered"

[[92]]
[1] "Simul_5000_5e-3_2_filtered"

[[93]]
[1] "Simul_5000_5e-3_3_filtered"

[[94]]
[1] "Simul_5000_5e-3_4_filtered"

[[95]]
[1] "Simul_5000_5e-3_5_filtered"

[[96]]
[1] "Simul_5000_5e-3_6_filtered"

[[97]]
[1] "Simul_5000_5e-3_7_filtered"

[[98]]
[1] "Simul_5000_

In [22]:
dataset_name_all=c('Splatter_5000_1_filtered')

In [24]:
for(dataset_name in dataset_name_all[1:length(dataset_name_all)]){
    print(dataset_name)
    
    start_time=as.numeric(Sys.time())
    
    seurat=read_in_seurat(dataset_name)
    
    seurat_hvg <- NormalizeData(seurat)

    all.genes <- rownames(seurat_hvg)
    seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

    seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

    seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))    
    seurat_hvg_pca_tsne <- RunTSNE(seurat_hvg_pca, dims= 1:10)    
    
    
    dat.expression<-seurat_hvg_pca[['RNA']]@counts
    
    dat.expression.median=apply(dat.expression,1,median)
    dat.detection <- dat.expression > dat.expression.median    
    
    dat.tsne <-Embeddings(seurat_hvg_pca_tsne, reduction = "tsne")
    dat.pca <-Embeddings(seurat_hvg_pca_tsne, reduction = "tsne")
    
    end_time=as.numeric(Sys.time())
    
    time_preprocessing=end_time-start_time
    
    start_time=as.numeric(Sys.time())
    res_2D <- haystack(dat.tsne, detection = dat.detection, method = "2D")
    end_time=as.numeric(Sys.time())
    time_2D=end_time-start_time
    
    start_time=as.numeric(Sys.time())
    res_highD <- haystack(dat.tsne, detection = dat.detection, method = "highD")
    end_time=as.numeric(Sys.time())
    time_highD=end_time-start_time
    
    write.table(res_2D$result[order(res_2D$result['log.p.adj']),],
                file=sprintf('datasets/extract/%s.hvg.ht.tsne.tsv',dataset_name),
                sep='\t',
                row.names=TRUE, 
                col.names=TRUE)   
    
    write.table(time_preprocessing+time_2D,
                file=sprintf('datasets/extract/%s.hvg.ht.tsne.runtime.tsv',dataset_name),
                sep='\t',
                row.names=FALSE, 
                col.names=FALSE)       
    
    write.table(res_highD$result[order(res_highD$result['log.p.adj']),],
                file=sprintf('datasets/extract/%s.hvg.ht.pca.tsv',dataset_name),
                sep='\t',
                row.names=TRUE, 
                col.names=TRUE) 
    
    write.table(time_preprocessing+time_highD,
                file=sprintf('datasets/extract/%s.hvg.ht.pca.runtime.tsv',dataset_name),
                sep='\t',
                row.names=FALSE, 
                col.names=FALSE)      
    
}

[1] "Splatter_5000_1_filtered"


Centering and scaling data matrix

PC_ 1 
Positive:  Gene1302, Gene1862, Gene2263, Gene4259, Gene4710, Gene2936, Gene992, Gene292, Gene3865, Gene1715 
	   Gene556, Gene688, Gene4448, Gene725, Gene1237, Gene2538, Gene2175, Gene1424, Gene912, Gene4637 
	   Gene56, Gene3426, Gene607, Gene1069, Gene1527, Gene895, Gene1362, Gene4604, Gene1572, Gene873 
Negative:  Gene2988, Gene1532, Gene4896, Gene2265, Gene1749, Gene4048, Gene4050, Gene154, Gene1976, Gene1319 
	   Gene3827, Gene930, Gene1825, Gene3656, Gene103, Gene3734, Gene4272, Gene3370, Gene4364, Gene302 
	   Gene211, Gene1400, Gene780, Gene4475, Gene443, Gene3997, Gene3280, Gene4919, Gene2603, Gene3974 
PC_ 2 
Positive:  Gene4904, Gene4197, Gene4679, Gene1736, Gene1715, Gene2987, Gene558, Gene4009, Gene977, Gene4923 
	   Gene378, Gene1625, Gene3816, Gene4085, Gene1112, Gene604, Gene3205, Gene2976, Gene4564, Gene650 
	   Gene2051, Gene897, Gene4746, Gene2341, Gene3077, Gene787, Gene2832, Gene397, Gene2175, Gene645 
Negative:  Gene3830, 

In [ ]:
aaa

In [11]:
res_2D

$results
           D_KL  log.p.vals     log.p.adj T.counts
2808 2.68398920  -5.1251452  -1.949053906      999
1217 0.00000000   0.0000000   0.000000000     1000
3969 0.00000000   0.0000000   0.000000000     1000
2244 0.00000000   0.0000000   0.000000000     1000
1469 0.00000000   0.0000000   0.000000000     1000
587  0.00000000   0.0000000   0.000000000     1000
703  0.00000000   0.0000000   0.000000000     1000
1101 0.00000000   0.0000000   0.000000000     1000
1866 2.68398920  -5.1251452  -1.949053906      999
1090 0.00000000   0.0000000   0.000000000     1000
3273 0.00000000   0.0000000   0.000000000     1000
1118 0.00000000   0.0000000   0.000000000     1000
2311 0.00000000   0.0000000   0.000000000     1000
3442 0.00000000   0.0000000   0.000000000     1000
2943 2.68398920  -5.1251452  -1.949053906      999
4256 0.00000000   0.0000000   0.000000000     1000
1268 0.00000000   0.0000000   0.000000000     1000
2415 2.68398920  -5.1251452  -1.949053906      999
448  2.68398920  -5.12

In [14]:
haystack(dat.tsne, detection = dat.detection, method = "2D")

### calling haystack_2D()...



ERROR: Error in haystack_2D(x[, dim1], x[, dim2], detection = detection, use.advanced.sampling = use.advanced.sampling, : The number of columns in 'detection' must be the same as the length of 'x'


In [22]:
dataset_name

[1] "Kohinbulk_filtered"

In [21]:
res_2D$result[order(res_2D$result['log.p.adj']),]

,D_KL,log.p.vals,log.p.adj,T.counts
,<dbl>,<dbl>,<dbl>,<int>
ENSG00000069482,0.8377246,-18.49607,-14.32893,267
ENSG00000006128,0.8421037,-18.14164,-13.97450,177
ENSG00000121966,0.8438000,-17.26487,-13.09773,217
ENSG00000168542,1.0390340,-17.16285,-12.99571,312
ENSG00000136110,0.6878173,-16.81286,-12.64572,268
ENSG00000146374,0.6926467,-16.79664,-12.62950,252
ENSG00000204335,0.6818064,-16.76082,-12.59368,256
ENSG00000205358,0.7682201,-16.68247,-12.51533,163
ENSG00000106038,0.7001881,-16.49548,-12.32834,172


In [20]:
res_highD$result[order(res_highD$result['log.p.adj']),]

,D_KL,log.p.vals,log.p.adj,T.counts
,<dbl>,<dbl>,<dbl>,<int>
ENSG00000121966,1.0071110,-27.63031,-23.46317,217
ENSG00000069482,1.0587212,-26.67426,-22.50712,267
ENSG00000254934,1.1477071,-26.23802,-22.07088,303
ENSG00000147869,0.9441783,-25.37861,-21.21147,221
ENSG00000204335,0.9904978,-25.25883,-21.09169,256
ENSG00000146374,0.9783368,-25.14600,-20.97886,252
ENSG00000128918,0.9086595,-24.05097,-19.88383,216
ENSG00000168542,1.1092045,-23.90070,-19.73356,312
ENSG00000136110,0.9771831,-23.84964,-19.68250,268


In [ ]:
|